In [25]:
import pandas as pd
df = pd.read_excel('Sample - Superstore.xls')

First, let us understand how many rows & columns we have using

In [26]:
df.shape

(9994, 21)

In [27]:
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


It seems the data contains really interesting information about the customer who has bought and the product that was sold. For our Sankey diagram, we want to visualize the sales per category & sub-category for the different customer segments.

So, let’s take a closer look at the values of the relevant columns. We start with the ‘Category’.

In [28]:
df.value_counts('Category')

Category
Office Supplies    6026
Furniture          2121
Technology         1847
Name: count, dtype: int64

Seems as if all sold products can be grouped into three main categories: Office Supplies, Furniture & Technology. While there are only these three main categories, there are much more sub-categories.

In [29]:
df.value_counts('Sub-Category')

Sub-Category
Binders        1523
Paper          1370
Furnishings     957
Phones          889
Storage         846
Art             796
Accessories     775
Chairs          617
Appliances      466
Labels          364
Tables          319
Envelopes       254
Bookcases       228
Fasteners       217
Supplies        190
Machines        115
Copiers          68
Name: count, dtype: int64

Overall we have 17 sub-categories each belonging to one of the three main categories. How about the customer segments?

In [30]:
df.value_counts('Segment')

Segment
Consumer       5191
Corporate      3020
Home Office    1783
Name: count, dtype: int64

We have three different customer segments — Consumers, Corporate and Home Office. But we don’t know yet how much they contributed to the overall Sales. We only know the number of sales transactions so far. Time to take a closer look at Sankey diagrams.

We will use Python Plotly to make our Sankey diagrams. In the beginning, the setup can be really confusing, so let me try to break it down.

First, let’s import Plotly.

In [31]:
import plotly.graph_objects as go

Let’s start with one very basic Sankey chart with three nodes and let me mark the important things as bold in the below code.

In [32]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad=15,
        thickness = 20,
        line = dict(color = 'black', width=0.5),
        label = ['nodel', 'node2','node3'],
        color = 'blue'),
    link = dict(source = [0,1],
                target = [2,2],
                value = [8,4])
    )
])
fig.update_layout(title_text = 'Supersore_Sales',font_size=10)
fig.show()

The first thing we need is our labels. We need to add all the values from columns ‘Segment’, ‘Category’, and ‘Sub-Category’ to our labels list. As we are lazy, we try to do it programmatically.

We read the unique values from our three relevant DataFrame columns into three lists:

segments
categories
subcategories

In [33]:
segments = df['Segment'].unique().tolist()
categories = df['Category'].unique().tolist()
subcategories = df['Sub-Category'].unique().tolist()
labels = segments+categories+subcategories
labels

['Consumer',
 'Corporate',
 'Home Office',
 'Furniture',
 'Office Supplies',
 'Technology',
 'Bookcases',
 'Chairs',
 'Labels',
 'Tables',
 'Storage',
 'Furnishings',
 'Art',
 'Phones',
 'Binders',
 'Appliances',
 'Paper',
 'Accessories',
 'Envelopes',
 'Fasteners',
 'Supplies',
 'Machines',
 'Copiers']

Next, we need our source & target indexes and their values. Let’s start with three empty lists.

In [34]:
sources=[]
targets=[]
values=[]

We are looping through our three segments, segment by segment. For each segment, we will loop through the subcategories. And for each subcategory, we will

reduce the DataFrame to the current segment only
reduce the DataFrame to the current subcategory only and sum up their overall sales
append the index of the segment to the sources array
append the index of the subcategory to the targets array
append the sum of the sales to the values array

In [35]:
for segment in segments:
    for subcat in subcategories:
        df1 = df.loc[df['Segment']==segment]
        df1 = df1.loc[df['Sub-Category']==subcat,'Sales'].sum()
        sources.append(labels.index(segment))
        targets.append(labels.index(subcat))
        values.append(df1)

Now we can forward our created lists to Plotly and already start plotting.

In [36]:
fig= go.Figure(data=[go.Sankey(
    node = dict(
        pad=15,
        thickness = 20,
        line = dict(color = 'black', width = 0.5),
        label = labels,
        color = 'blue'
    
    ),
    link = dict(
        source = sources,
        target  = targets,
        value = values
    )
)])

fig.update_layout(title_text = 'Superstore Sales', font_size = 15)
fig.show()

But the third node-level of main categories is still missing. So let’s go one step back and let’s start looping again.

You already know the first loop from the last step. Nothing has changed here. But we have a second loop. Here we do the below things

In [37]:
for segment in segments:
        for subcat in subcategories:
            df1 = df.loc[df['Segment']==segment]
            df1 = df1.loc[df['Sub-Category']==subcat,'Sales'].sum()
            sources.append(labels.index(segment))
            targets.append(labels.index(subcat))
            values.append(df1)

for categorie in categories:
    for subcat in df['Sub-Category'].unique():
        df2 = df.loc[df['Category']==categorie]
        df2 = df2.loc[df['Sub-Category']==subcat,'Sales'].sum()
        sources.append(labels.index(subcat))
        targets.append(labels.index(categorie))
        values.append(df2)

We are looping through our three main categories, category by category. For each main category, we will loop through the subcategories. And for each subcategory, we will

- reduce the DataFrame to the current category only
- reduce the DataFrame to the current subcategory only and sum up their overall sales
- append the index of the subcategory to the sources array
- append the index of the main category to the targets array
- append the sum of the sales to the values array

Done. And we all know what this means! Plotting time.

In [38]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = "blue"
    ),
    link = dict(
      source = sources,
      target = targets,
      value = values
  ))])

fig.update_layout(title_text="Superstore Sales", font_size=10)
fig.show()